# Leave-One-Subject-Out Evaluation

In [1]:
import numpy as np
import cv2
import dlib
from imutils import face_utils
import glob
import pickle
from random import shuffle
from sklearn.model_selection import train_test_split
from sklearn import metrics
import json
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import yaml

### 3D Model points for SolvePnP
We will approximate the 3D points of the following face parts with the correspoinding coordinates. This is a general model of the human face and we do not need to worry much about absolute accuracy.

In [2]:
def generate_solvepnp_parameters(size, landmarks):
    focal_length = size[1]
    center = (size[1]/2, size[0]/2)
    camera_matrix = np.array(
                             [[focal_length, 0, center[0]],
                             [0, focal_length, center[1]],
                             [0, 0, 1]], dtype = "double"
                             )

    # Grab the 2D coordinates of our six sample points
    image_points = np.array([
        (landmarks[33]['x'], landmarks[33]['y']) ,     # Nose tip
        (landmarks[8]['x'], landmarks[8]['y']),     # Chin
        (landmarks[36]['x'], landmarks[36]['y']),     # Left eye left corner
        (landmarks[45]['x'], landmarks[45]['y']),     # Right eye right corner
        (landmarks[48]['x'], landmarks[48]['y']),     # Left Mouth corner
        (landmarks[54]['x'], landmarks[54]['y'])      # Right mouth corner
    ], dtype="double")
    
    return image_points, camera_matrix

# 3D model points.
model_points = np.array([
                            (0.0, 0.0, 0.0),             # Nose tip
                            (0.0, -330.0, -65.0),        # Chin
                            (-225.0, 170.0, -135.0),     # Left eye left corner
                            (225.0, 170.0, -135.0),      # Right eye right corne
                            (-150.0, -150.0, -125.0),    # Left Mouth corner
                            (150.0, -150.0, -125.0)      # Right mouth corner
                         
                        ])

dist_coeffs = np.zeros((4,1)) # Assuming no lens distortion

In [20]:
def visualize_image(im, rotation_vector, translation_vector, image_points, camera_matrix):
    # Project a 3D point (0, 0, 1000.0) onto the image plane.
    # We use this to draw a line sticking out of the nose
    (nose_end_point2D, jacobian) = cv2.projectPoints(
        np.array([(0.0, 0.0, 500.0)]), rotation_vector, translation_vector, camera_matrix, dist_coeffs
        )
    for p in image_points:
        cv2.circle(im, (int(p[0]), int(p[1])), 3, (0,0,255), -1)
    cv2.circle(im, (int(iris_left[0]), int(iris_left[1])), 3, (0, 0, 255), -1)
    cv2.circle(im, (int(iris_right[0]), int(iris_right[1])), 3, (0, 0, 255), -1)
    p1 = ( int(image_points[0][0]), int(image_points[0][1]) )
    p2 = ( int(nose_end_point2D[0][0][0]), int(nose_end_point2D[0][0][1]) )

    # Draw a line connecting the two points. This line must show
    # the direction out of the nose
    cv2.line(im, p1, p2, (255,0,0), 2)
    # Display image
    cv2.imshow(output, im)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

***

In [3]:
participants = glob.glob('dataset/*')
print(len(participants))

53


In [4]:
# Number of training examples to use(0-2758)
DATASET_SIZE = 2758
DEBUG = False

# Load the dataset
with open('data_cleaned.json') as json_file:
    data_all = json.load(json_file)
# Extract the keys in sorted order
keys_all = sorted(data_all)
# Convert python list to np array
keys_all = np.asarray(keys_all)
print(len(keys_all))

2758


In [7]:
# Accuracy metrics for the whole dataset. These are compouter
# by leaving every Subject out one time, calculating the accuracy for each
# one and then taking the mean.
rf_accuracy = 0
svm_accuracy = 0

# Array to keep track of subjects with low score
low_score_subjects_rf = []
low_score_subjects_svm = []

for j in range(len(participants)):
    uuid_excluded = participants[j].split('/')[1]

    indices_excluded = []
    keys_excluded = []
    for i in range(DATASET_SIZE):
        key = keys_all[i]
        uuid = key.split('/')[0]
        if(uuid == uuid_excluded):
            indices_excluded.append(i)
            keys_excluded.append(key)
    keys = np.delete(keys_all, indices_excluded)

    CURRENT_DATASET_SIZE = keys.shape[0]

    X = np.zeros((CURRENT_DATASET_SIZE, 14, 1))
    y = np.zeros(CURRENT_DATASET_SIZE)

    # Indices that the SolvePnP failed
    failed_indices = []

    for i in range(CURRENT_DATASET_SIZE):
        key = keys[i]

        # Approximate camera intrinsic parameters
#         im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
#         size = im.shape
        with open('dataset/' + key.split('/')[0] + '/data.yml', 'r') as stream:
            try:
                laptop = yaml.safe_load(stream)['laptop']
                if(laptop == 'k' or laptop == 'K'):
                    size = (480, 640, 3)
                elif(laptop == 'c' or laptop == 'C'):
                    size = (720, 1280, 3)
            except yaml.YAMLError as exc:
                print(exc)

        landmarks = data_all[key]['landmarks']
        
        image_points, camera_matrix = generate_solvepnp_parameters(size, landmarks)

        # Solve the PnP problem with the parameters specified above
        # and obtain rotation and translation vectors
        (success, rotation_vector, translation_vector) = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_DLS
            )
        
        # Data from Architecture #2. Reshape is done for compatibility reasons
        iris_right = np.reshape(np.asarray(data_all[key]['iris_right']), (2, 1))
        iris_left = np.reshape(np.asarray(data_all[key]['iris_left']), (2, 1))

        # Data from Architecture #3
        left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
        right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

        X[i, :] = np.concatenate((rotation_vector, translation_vector, iris_left, iris_right, 
                                 left_vector, right_vector), axis=0)

        # Check if it is positive or negative example
        output = key.split('/')[1]
        if(output == 'positive'):
            y[i] = 1
        elif(output == 'negative'):
            y[i] = 0
    
        # Remove examples that SolvePnP crashed
        if(X[i, 0] > 10000):
            print(key)
            failed_indices.append(i)
    X = np.delete(X, failed_indices, axis=0)
    y = np.delete(y, failed_indices, axis=0)
    
    X = X.squeeze()

    m = X.mean(axis=0)
    std = X.std(axis=0)
    
    X_scaled = (X - m)/std

    ### Train and Predict

#     X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.1)
    X_train, y_train = X_scaled, y

    rf_classifier = RandomForestClassifier(n_estimators=500, random_state=1)
    rf_classifier.fit(X_train, y_train)
#     y_pred_rf = rf_classifier.predict(X_test)

#     print('Test set accuracy for Random Forest: ', metrics.accuracy_score(y_test, y_pred_rf))

    svm_classifier = svm.SVC(C=10, kernel='rbf', gamma='auto')
    svm_classifier.fit(X_train, y_train)
#     y_pred_svm = svm_classifier.predict(X_test)

#     print('Training set accuracy for SVM:', svm_classifier.score(X_train, y_train))
#     print('Test set accuracy for SVM: ', metrics.accuracy_score(y_test, y_pred_svm))

    X_eval = np.zeros((len(keys_excluded), 14, 1))
    y_eval = np.zeros(len(keys_excluded))

    for i in range(len(keys_excluded)):
        key = keys_excluded[i]

        # Approximate camera intrinsic parameters
#         im = cv2.imread('dataset/' + key)   # This imread is time consuming! Another way?
#         size = im.shape
        with open('dataset/' + key.split('/')[0] + '/data.yml', 'r') as stream:
            try:
                laptop = yaml.safe_load(stream)['laptop']
                if(laptop == 'k' or laptop == 'K'):
                    size = (480, 640, 3)
                elif(laptop == 'c' or laptop == 'C'):
                    size = (720, 1280, 3)
            except yaml.YAMLError as exc:
                print(exc)

        landmarks = data_all[key]['landmarks']
        
        image_points, camera_matrix = generate_solvepnp_parameters(size, landmarks)

        # Solve the PnP problem with the parameters specified above
        # and obtain rotation and translation vectors
        (success, rotation_vector, translation_vector) = cv2.solvePnP(
            model_points, image_points, camera_matrix, dist_coeffs, flags=cv2.SOLVEPNP_DLS
            )

        # Data from Architecture #2. Reshape is done for compatibility reasons
        iris_right = np.reshape(np.asarray(data_all[key]['iris_right']), (2, 1))
        iris_left = np.reshape(np.asarray(data_all[key]['iris_left']), (2, 1))

        # Data from Architecture #3
        left_vector = np.asarray( (abs(iris_left[0] - landmarks[39]['x']), abs(iris_left[1] - landmarks[39]['y'])) )
        right_vector = np.asarray( (abs(iris_right[0] - landmarks[42]['x']), abs(iris_right[1] - landmarks[42]['y'])) )

        X_eval[i, :] = np.concatenate((rotation_vector, translation_vector, iris_left, iris_right, 
                                      left_vector, right_vector), axis=0)

        # Check if it is positive or negative example
        output = key.split('/')[1]
        if(output == 'positive'):
            y_eval[i] = 1
        elif(output == 'negative'):
            y_eval[i] = 0

    X_eval = X_eval.squeeze()
#     print(X_eval.shape, y_eval.shape)

    # Normalization
    m_eval = X_eval.mean(axis=0)
    std_eval = X_eval.std(axis=0)
    X_eval = (X_eval - m_eval)/std_eval

    # Predict Random Forest
    y_eval_rf = rf_classifier.predict(X_eval)
    rf_accuracy_subject = metrics.accuracy_score(y_eval, y_eval_rf)
    print('Test set accuracy for subject {} with Random Forest: {}'.format(j, rf_accuracy_subject))

    # Predict SVM
    y_pred_svm = svm_classifier.predict(X_eval)
    svm_accuracy_subject = metrics.accuracy_score(y_eval, y_pred_svm)
    print('Test set accuracy for subject {} with SVM: {}'.format(j, svm_accuracy_subject))
    
    if(rf_accuracy_subject <= 0.5):
        low_score_subjects_rf.append(uuid_excluded)
    if(svm_accuracy_subject <= 0.5):
        low_score_subjects_svm.append(uuid_excluded)
    
    rf_accuracy += rf_accuracy_subject*len(keys_excluded)
    svm_accuracy += svm_accuracy_subject*len(keys_excluded)

rf_accuracy = rf_accuracy/DATASET_SIZE
svm_accuracy = svm_accuracy/DATASET_SIZE
print(rf_accuracy)
print(svm_accuracy)

Test set accuracy for subject 0 with Random Forest: 0.6363636363636364
Test set accuracy for subject 0 with SVM: 0.6666666666666666
Test set accuracy for subject 1 with Random Forest: 0.9032258064516129
Test set accuracy for subject 1 with SVM: 0.7741935483870968
Test set accuracy for subject 2 with Random Forest: 0.8181818181818182
Test set accuracy for subject 2 with SVM: 0.6818181818181818
Test set accuracy for subject 3 with Random Forest: 0.46153846153846156
Test set accuracy for subject 3 with SVM: 0.3076923076923077
Test set accuracy for subject 4 with Random Forest: 0.0
Test set accuracy for subject 4 with SVM: 0.3333333333333333
Test set accuracy for subject 5 with Random Forest: 0.5
Test set accuracy for subject 5 with SVM: 0.625
Test set accuracy for subject 6 with Random Forest: 0.7474747474747475
Test set accuracy for subject 6 with SVM: 0.7272727272727273
Test set accuracy for subject 7 with Random Forest: 0.7727272727272727
Test set accuracy for subject 7 with SVM: 0.477

In [18]:
print(len(low_score_subjects_rf))
print(len(low_score_subjects_svm))

11
14


In [19]:
file1 = open("low_score_rf.txt", "w") 
for i in low_score_subjects_rf:
    file1.write(i + '\n')
    
file2 = open("low_score_svm.txt", "w")
for i in low_score_subjects_svm:
    file2.write(i + '\n')